## Create datasets for the Content-based Filter

This notebook builds the data we will use for creating our content based model. We'll collect the data via a collection of SQL queries from the publicly available Kurier.at dataset in BigQuery.
Kurier.at is an Austrian newsite. The goal of these labs is to recommend an article for a visitor to the site. In this lab we collect the data for training, in the subsequent notebook we train the recommender model. 

This notebook illustrates
* how to pull data from BigQuery table and write to local files
* how to make reproducible train and test splits 

In [1]:
import os
import tensorflow as tf
import numpy as np
from google.cloud import bigquery 

PROJECT = 'qwiklabs-gcp-01-493597224b34' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'qwiklabs-gcp-01-493597224b34' # REPLACE WITH YOUR BUCKET NAME
REGION = 'australia-southeast1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1'

In [2]:
%%bash
gcloud  config  set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


We will use this helper function to write lists containing article ids, categories, and authors for each article in our database to local file. 

In [3]:
def write_list_to_disk(my_list, filename):
    with open(filename, 'w') as f:
        for item in my_list:
            line = "%s\n" % item
            f.write(line)

### Pull data from BigQuery

The cell below creates a local text file containing all the article ids (i.e. 'content ids') in the dataset. 

Have a look at the original dataset in [BigQuery](https://console.cloud.google.com/bigquery?p=cloud-training-demos&d=GA360_test&t=ga_sessions_sample). Then read through the query below and make sure you understand what it is doing. 

In [4]:
sql="""
#standardSQL

SELECT (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id 
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`, UNNEST(hits) AS hits
WHERE hits.type = "PAGE"    # only include hits on pages
AND (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY content_id
"""

content_ids_list = bigquery.Client().query(sql).to_dataframe()['content_id'].tolist()
write_list_to_disk(content_ids_list, "content_ids.txt")

print(f"Some sample content IDs {content_ids_list[:3]}")
print(f"The total number of articles is {len(content_ids_list)}")

Some sample content IDs ['299824032', '299865757', '299918857']
The total number of articles is 15634


There should be 15,634 articles in the database.  

Next, we'll create a local file which contains a list of article categories and a list of article authors.

Note the change in the index when pulling the article category or author information.
Refer back to the original dataset, use the `hits.customDimensions.index` field to verify the correct index.	 

In [5]:
sql="""
#standardSQL
SELECT (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`, UNNEST(hits) AS hits
WHERE hits.type = "PAGE"   # only include hits on pages
AND (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY category
"""

categories_list = bigquery.Client().query(sql).to_dataframe()['category'].tolist()
write_list_to_disk(categories_list, "categories.txt")

print(categories_list)

['Lifestyle', 'News', 'Stars & Kultur']


The categories are 'News', 'Stars & Kultur', and 'Lifestyle'.  

Now for the authot list.  
When creating the author list, we'll only use the first author information for each article.   

In [6]:
sql="""
#standardSQL
SELECT REGEXP_EXTRACT((SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)), r"^[^,]+")  AS first_author  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`, UNNEST(hits) AS hits
WHERE hits.type = "PAGE"   # only include hits on pages
AND (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY first_author
"""

authors_list = bigquery.Client().query(sql).to_dataframe()['first_author'].tolist()
write_list_to_disk(authors_list, "authors.txt")

print(f"Some sample authors {authors_list[:10]}")
print(f"The total number of authors is {len(authors_list)}")

Some sample authors ['Christina Michlits', 'Mathias Kainz', 'Thomas  Trescher', 'Stefan Berndl', 'Anita Kattinger', 'Martina Salomon', 'Marlene Patsalidis', 'Georg Leyrer', 'Elisabeth Spitzer', 'Elisabeth Sereda']
The total number of authors is 385


There should be 385 authors in the database. 

### Create train and test sets.

In this section, we will create the train/test split of our data for training our model.  
We use the concatenated values for visitor id and content id to create a _farm fingerprint_, taking approximately 90% of the data for the training set and 10% for the test set.

In [7]:
sql="""
WITH site_history as (
  SELECT fullVisitorId as visitor_id,
    (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
    (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
    (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
    (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
    SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
    LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM `cloud-training-demos.GA360_test.ga_sessions_sample`, UNNEST(hits) AS hits
  WHERE hits.type = "PAGE" # only include hits on pages
  AND fullVisitorId IS NOT NULL
  AND hits.time != 0
  AND hits.time IS NOT NULL
  AND (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT visitor_id, content_id, category, REGEXP_REPLACE(title, r",", "") as title, REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970, 1, 1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL
AND ABS(MOD(FARM_FINGERPRINT(CONCAT(visitor_id, content_id)), 10)) < 9
"""

training_set_df = bigquery.Client().query(sql).to_dataframe()
training_set_df.to_csv('training_set.csv', header=False, index=False, encoding='utf-8')
training_set_df.head(7)

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1030878773401944300,299974496,News,Kurier TV-News: Die Baustelle Bildung,Stefan Berndl,574,299830996
1,1030878773401944300,299830996,News,Wie die Schule in der Neuzeit ankommen könnte,Martina Salomon,574,299901255
2,1045356747303546594,299792812,News,Bundesliga: Kein Videobeweis beim Schlager Rap...,None,574,299779564
3,1045356747303546594,299779564,Stars & Kultur,Geschenk: Nicole Kidman bekommt Traumhaus um 4...,Elisabeth Spitzer,574,299809748
4,104581626240810883,299982579,News,VIDEO: Basejumper springen von Berg in Flugzeug,Mathias Kainz,574,299935287
5,1065940605821174786,299173332,Stars & Kultur,Verändert: So sehen die 80er-Stars heute aus,Christina Michlits,574,299775963
6,1065940605821174786,299775963,News,Trump: Der letzte Verteidiger von Roy Moore,Thomas Trescher,574,299798467


In [8]:
sql="""
WITH site_history as (
  SELECT fullVisitorId as visitor_id,
    (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
    (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
    (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
    (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
    SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
    LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM `cloud-training-demos.GA360_test.ga_sessions_sample`, UNNEST(hits) AS hits
  WHERE hits.type = "PAGE"  # only include hits on pages
  AND fullVisitorId IS NOT NULL
  AND hits.time != 0
  AND hits.time IS NOT NULL
  AND (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT visitor_id, content_id, category, REGEXP_REPLACE(title, r",", "") as title, REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL
AND ABS(MOD(FARM_FINGERPRINT(CONCAT(visitor_id, content_id)), 10)) >= 9
"""

test_set_df = bigquery.Client().query(sql).to_dataframe()
test_set_df.to_csv('test_set.csv', header=False, index=False, encoding='utf-8')
test_set_df.head(7)

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1415632513742447264,299777722,Stars & Kultur,"Willow Smith: ""Es ist schrecklich so aufzuwach...",Elisabeth Spitzer,574,299777082
1,171240914473388761,299831571,News,Chinesen investieren 3 Mrd. Euro in Osteuropa,Peter Temel,574,299915880
2,1917946462475400228,77853842,News,Mängel dokumentieren und Streit vermeiden,Ursula Horvath,535,284769185
3,1917946462475400228,284769185,News,Nachfolger gesucht: Koller vor dem Abgang,Andreas Heidenreich,572,287515242
4,1917946462475400228,223645183,News,Gerümpel im Hausflur: Wer zahlt die Entsorgung?,Claudia Elmer,561,223645183
5,1917946462475400228,223645183,News,Gerümpel im Hausflur: Wer zahlt die Entsorgung?,Claudia Elmer,561,273295418
6,1917946462475400228,223645183,News,Gerümpel im Hausflur: Wer zahlt die Entsorgung?,Claudia Elmer,561,284332710


Let's have a look at the two csv files we just created containing the training and test set. We'll also do a line count of both files to confirm that we have achieved an approximate 90/10 train/test split.  
In the next notebook, **Content Based Filtering** we will build a model to recommend an article given information about the current article being read, such as the category, title, author, and publish date. 

In [9]:
%%bash
wc -l *_set.csv

   25599 test_set.csv
  232308 training_set.csv
  257907 total


In [10]:
!head -7 *_set.csv

==> test_set.csv <==
1415632513742447264,299777722,Stars & Kultur,"Willow Smith: ""Es ist schrecklich so aufzuwachsen""",Elisabeth Spitzer,574,299777082
171240914473388761,299831571,News,Chinesen investieren 3 Mrd. Euro in Osteuropa,Peter Temel,574,299915880
1917946462475400228,77853842,News,Mängel dokumentieren und Streit vermeiden,Ursula Horvath,535,284769185
1917946462475400228,284769185,News,Nachfolger gesucht: Koller vor dem Abgang,Andreas Heidenreich,572,287515242
1917946462475400228,223645183,News,Gerümpel im Hausflur: Wer zahlt die Entsorgung?,Claudia Elmer,561,223645183
1917946462475400228,223645183,News,Gerümpel im Hausflur: Wer zahlt die Entsorgung?,Claudia Elmer,561,273295418
1917946462475400228,223645183,News,Gerümpel im Hausflur: Wer zahlt die Entsorgung?,Claudia Elmer,561,284332710

==> training_set.csv <==
1030878773401944300,299974496,News,Kurier TV-News: Die Baustelle Bildung,Stefan Berndl,574,299830996
1030878773401944300,299830996,News,Wie die Schule in der Neuzeit 